In [1]:
import pandas as pd
import numpy as np
import string
import re

from crontab import CronTab

### Scheduler - Only run it once in your local machine
Purpose: to create the cron job once in your local machine

In [2]:
# # scheduling tasks in python
# cron = CronTab(user=True)
# job = cron.new(command='python main.py')

# job.hour.on(1)
# job.minute.on(0)
# cron.write()

### Read CSV files

In [3]:
users_df = pd.read_csv('inputs/users.csv')
users_df

,id,email,full_name,gender
0,59866fb8840fe0000b234493,xking2683@yahoo.com,Robert Sanders,M
1,59867137078d11000a1462e2,jeffery332612@gmail.com,Michael Stone,F
2,598678c9840fe0000b2344b1,catherine142498@gmail.com,Henry Nelson,F
3,59867b37078d11000a1462fb,uparker2499@humphrey.org,Stacy Thomas,M
4,59867ba8078d11000a146300,nathanbrown2559@yahoo.com,Sarah Avila,F
...,...,...,...,...
95,5992ea3ca471e5000be04a7d,palmeranthony3008@yahoo.com,Morgan Lee,M
96,5992ea42a471e5000be04a81,kevingarcia3030@yahoo.com,Destiny Chavez,M
97,5992edb75afffc0009f2e325,reginaldstone3000@keller-morton.com,Gregory Gonzalez,F
98,5992f0195afffc0009f2e332,hebertbrett3048@jones.com,Wesley Wade,F


In [4]:
goals_df = pd.read_csv('inputs/goals.csv')
goals_df

,Goal Type,User ID,Portfolio ID,Region,ID
0,GENERAL_INVESTING,SG59866fb8840fe0000b234493,SGa09326d4-178f-482c-aaf3-ab8aab32b3c6,SG,598670d5078d11000a1462dd
1,RETIREMENT,SG598678c9840fe0000b2344b1,SGb1df85ed-a4dd-49ea-aa5f-c8149c6f0130,SG,5986794f840fe0000b2344b5
2,GENERAL_INVESTING,SG59867b37078d11000a1462fb,SG2a91b4b8-7d8d-4af4-9908-6ca6ad05e846,SG,59867b82840fe0000b2344bd
3,RETIREMENT,SG59867ba8078d11000a146300,SG82639fba-4080-478a-beba-d4e44e7c57ad,SG,59867d5d078d11000a146309
4,CHILD_EDUCATION,SG59867dd0078d11000a146310,SG7b1efdaa-94a7-4df0-adb9-cbe9c0f81042,SG,59867e50840fe0000b2344c8
...,...,...,...,...,...
505,GENERAL_INVESTING,SG598699c8078d11000a14635d,SG1f6255de-fc0a-4b67-adfb-c3a3d285c162,SG,60b7621a2291f1c9396e105b
506,THEME_TECH,SG599042f55afffc0009f2e034,SGe1d83dc5-9429-45f9-85ba-097155a5da41,SG,615f35f481c22b4664c6af31
507,THEME_CONSUMER,SG599042f55afffc0009f2e034,SGb9144ad9-14ce-48bb-ac6b-92ed489de533,SG,615f362a5032af93a53de2c2
508,THEME_HEALTHCARE,SG599042f55afffc0009f2e034,SGce402ae1-6217-4ff1-b3fd-5693cd5b3ef2,SG,615f36715032afaa683de2d0


In [5]:
perf_df = pd.read_csv('inputs/performance.csv')
perf_df

,portfolioid,calcdate,officialnav,currency
0,SG14a674a8-c456-4af8-b484-0b68b11359fc,1.637626e+09,1028.44,SGD
1,SG14a674a8-c456-4af8-b484-0b68b11359fc,1.637539e+09,1028.44,SGD
2,SG14a674a8-c456-4af8-b484-0b68b11359fc,1.637453e+09,1028.35,SGD
3,SG14a674a8-c456-4af8-b484-0b68b11359fc,1.637366e+09,1028.35,SGD
4,SG14a674a8-c456-4af8-b484-0b68b11359fc,1.637280e+09,1028.35,SGD
...,...,...,...,...
230,SGc6aef4ca-6f53-4c1a-8896-055ee1d57866,1.636502e+09,1138.16,SGD
231,SGc6aef4ca-6f53-4c1a-8896-055ee1d57866,1.636589e+09,1138.27,SGD
232,SGc6aef4ca-6f53-4c1a-8896-055ee1d57866,1.636762e+09,1138.37,SGD
233,SGc6aef4ca-6f53-4c1a-8896-055ee1d57866,1.636675e+09,1138.37,SGD


### Data Preprocessing

In [6]:
# Common Method

def standardise_column_names(df, remove_punct=True):
    """ 
    Converts all column names to lower case replacing
    whitespace of any length with a single underscore.
    Also, remove punctuations if included.
    
    """

    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

    for c in df.columns:
        
        c_mod = c.lower()
        
        if remove_punct:            
            c_mod = c_mod.translate(translator)
        
        c_mod = '_'.join(c_mod.split(' '))
        
        if c_mod[-1] == '_':
            c_mod = c_mod[:-1]
        
        c_mod = re.sub(r'\_+', '_', c_mod)
        
        df.rename({c: c_mod}, inplace=True, axis=1)

    return df

In [7]:
# Standardise all columns names

users_df = standardise_column_names(users_df)
goals_df = standardise_column_names(goals_df)
perf_df = standardise_column_names(perf_df)

In [8]:
### Users Data

# Split first name and last name first
users_df[['first_name','last_name']] = users_df['full_name'].loc[users_df['full_name'].str.split().str.len() == 2].str.split(expand=True)

# drop email columns
users_df = users_df[['id', 'first_name', 'last_name', 'gender']]
users_df

,id,first_name,last_name,gender
0,59866fb8840fe0000b234493,Robert,Sanders,M
1,59867137078d11000a1462e2,Michael,Stone,F
2,598678c9840fe0000b2344b1,Henry,Nelson,F
3,59867b37078d11000a1462fb,Stacy,Thomas,M
4,59867ba8078d11000a146300,Sarah,Avila,F
...,...,...,...,...
95,5992ea3ca471e5000be04a7d,Morgan,Lee,M
96,5992ea42a471e5000be04a81,Destiny,Chavez,M
97,5992edb75afffc0009f2e325,Gregory,Gonzalez,F
98,5992f0195afffc0009f2e332,Wesley,Wade,F


In [9]:
### Goals Data

# Create new column: strategy_id, values from goal_type
goal_mapping = {
    "GENERAL_INVESTING": "CORE",
    "RETIREMENT": "CORE",
    "CHILD_EDUCATION": "CORE",
    "BUY_A_HOME": "CORE",
    "EMERGENCY_FUND": "CORE",
    "TRAVEL": "CORE",
    "WEDDING": "CORE",
    "START_BUSINESS": "CORE",
    "THEME_TECH": "THEMATIC",
    "THEME_CONSUMER": "THEMATIC",
    "THEME_HEALTHCARE": "THEMATIC",
    "INCOME": "INCOME_SGD",
    "MMF": "MMF_SGD",
    "VEHICLE": "CORE" 
}

goals_df['strategy_id'] = goals_df['goal_type'].apply(lambda x: goal_mapping[x])
goals_df['portfolio_id'] = goals_df['region'] + goals_df['portfolio_id']
goals_df

,goal_type,user_id,portfolio_id,region,id,strategy_id
0,GENERAL_INVESTING,SG59866fb8840fe0000b234493,SGSGa09326d4-178f-482c-aaf3-ab8aab32b3c6,SG,598670d5078d11000a1462dd,CORE
1,RETIREMENT,SG598678c9840fe0000b2344b1,SGSGb1df85ed-a4dd-49ea-aa5f-c8149c6f0130,SG,5986794f840fe0000b2344b5,CORE
2,GENERAL_INVESTING,SG59867b37078d11000a1462fb,SGSG2a91b4b8-7d8d-4af4-9908-6ca6ad05e846,SG,59867b82840fe0000b2344bd,CORE
3,RETIREMENT,SG59867ba8078d11000a146300,SGSG82639fba-4080-478a-beba-d4e44e7c57ad,SG,59867d5d078d11000a146309,CORE
4,CHILD_EDUCATION,SG59867dd0078d11000a146310,SGSG7b1efdaa-94a7-4df0-adb9-cbe9c0f81042,SG,59867e50840fe0000b2344c8,CORE
...,...,...,...,...,...,...
505,GENERAL_INVESTING,SG598699c8078d11000a14635d,SGSG1f6255de-fc0a-4b67-adfb-c3a3d285c162,SG,60b7621a2291f1c9396e105b,CORE
506,THEME_TECH,SG599042f55afffc0009f2e034,SGSGe1d83dc5-9429-45f9-85ba-097155a5da41,SG,615f35f481c22b4664c6af31,THEMATIC
507,THEME_CONSUMER,SG599042f55afffc0009f2e034,SGSGb9144ad9-14ce-48bb-ac6b-92ed489de533,SG,615f362a5032af93a53de2c2,THEMATIC
508,THEME_HEALTHCARE,SG599042f55afffc0009f2e034,SGSGce402ae1-6217-4ff1-b3fd-5693cd5b3ef2,SG,615f36715032afaa683de2d0,THEMATIC


In [10]:
from datetime import datetime
import tzlocal

def convert_to_local_time(value):
    unix_timestamp = float(value)
    local_timezone = tzlocal.get_localzone() # get pytz timezone
    local_time = datetime.fromtimestamp(unix_timestamp, local_timezone)
    
    local_time = local_time.strftime("%m,%d,%Y,%H:%M")
    return local_time

In [11]:
### Performance Data

perf_df.rename(columns={"portfolioid": "portfolio_id", "calcdate": "calc_date", "officialnav": "official_nav"}, inplace=True)
perf_df['calc_date'] = perf_df['calc_date'].apply(lambda x: convert_to_local_time(x))
perf_df['above_1000'] = perf_df['official_nav'].apply(lambda x: True if x > 1000 else False )
perf_df

,portfolio_id,calc_date,official_nav,currency,above_1000
0,SG14a674a8-c456-4af8-b484-0b68b11359fc,"11,23,2021,08:00",1028.44,SGD,True
1,SG14a674a8-c456-4af8-b484-0b68b11359fc,"11,22,2021,08:00",1028.44,SGD,True
2,SG14a674a8-c456-4af8-b484-0b68b11359fc,"11,21,2021,08:00",1028.35,SGD,True
3,SG14a674a8-c456-4af8-b484-0b68b11359fc,"11,20,2021,08:00",1028.35,SGD,True
4,SG14a674a8-c456-4af8-b484-0b68b11359fc,"11,19,2021,08:00",1028.35,SGD,True
...,...,...,...,...,...
230,SGc6aef4ca-6f53-4c1a-8896-055ee1d57866,"11,10,2021,08:00",1138.16,SGD,True
231,SGc6aef4ca-6f53-4c1a-8896-055ee1d57866,"11,11,2021,08:00",1138.27,SGD,True
232,SGc6aef4ca-6f53-4c1a-8896-055ee1d57866,"11,13,2021,08:00",1138.37,SGD,True
233,SGc6aef4ca-6f53-4c1a-8896-055ee1d57866,"11,12,2021,08:00",1138.37,SGD,True
